In [1]:
import re
import time
from tqdm import tqdm
 
import numpy as np
import pandas as pd
 

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from selenium import webdriver

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from pymystem3 import Mystem
from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import *

In [2]:
text = ['поиск ответ начинаться исследование интервью отзыв поэтому работник додо это буквально автор герой ролик который посвящать текст спасибо даша весь смена пиццерия вышний волочек то готовить клиент вынос при сниматься ролик спасибо мухаммед москва то приезжать два смена помимо свой два смена ради эпизод чат психологический поддержка спасибо миша екатеринбург армен москва совместный работа адаптация приложение додо незрячий спасибо тренер роман петербург история то сложно угощать ребенок выезд спасибо дарья челябинск благодаря узнавать додстер удобный что протекать рюкзак ольга спасибо отзыв наклейка поменяться много должный портить маникюр кирилл петрозаводск наблюдательность лампа новый интерьер другой андрей волгоград настойчивость темный тема то возможно спасибо все не забывать полагать соус спасибо что принимать и напоминать соус забывать мы вы любить ,', 'годный однако ,', 'офигенный ,', 'прекрасный клип додо хороший пицца ,', 'отличный ролл ,', 'круто ,', 'прекрасно удача ,', 'чело темный тема отвечать ж печально уже лампа делать высокий новый пиццерия сложно все нормальный сделать также суть про психологический поддержка сказать ,', 'ороший ролик вкусный пицца ,', 'додо родной обожать любить весь сердце ,', 'блин классный ролик когда думать то сколько получать самоотверженный человек который вы работать становиться грустно ,', 'нужный этот темный тема если отдельный пункт настройка то так некоторый сайт вкладка приложение она быть это безумно бить глаз долго минута не выдерживать ,', 'додо спасибо пицца за лампа ,', 'додо любить пицца вкусный спасибо готовить я для ,', 'очень классный ролик задумываться будущее полнометражный картина ,', 'какой роль занимать жизнь человек намеренно коверкать русский язык просто необразованный какой роль занимать какой играть значение ,', 'обожать додо пицца какой город не приезжать первый дело искать ваш ресторан крутой ,', 'жаль качество ингридиент смочь удерживать уровень тут понятно что дело чизкейк любимый у ,', 'очень понравиться ролик смысл что не привереда ,', 'интересно какой я шанс выйгрывать учавствовать еще человек ,']
for _ in range(10):

    print(text[np.random.randint(len(text))])

чело темный тема отвечать ж печально уже лампа делать высокий новый пиццерия сложно все нормальный сделать также суть про психологический поддержка сказать ,
блин классный ролик когда думать то сколько получать самоотверженный человек который вы работать становиться грустно ,
нужный этот темный тема если отдельный пункт настройка то так некоторый сайт вкладка приложение она быть это безумно бить глаз долго минута не выдерживать ,
жаль качество ингридиент смочь удерживать уровень тут понятно что дело чизкейк любимый у ,
прекрасно удача ,
додо любить пицца вкусный спасибо готовить я для ,
круто ,
офигенный ,
поиск ответ начинаться исследование интервью отзыв поэтому работник додо это буквально автор герой ролик который посвящать текст спасибо даша весь смена пиццерия вышний волочек то готовить клиент вынос при сниматься ролик спасибо мухаммед москва то приезжать два смена помимо свой два смена ради эпизод чат психологический поддержка спасибо миша екатеринбург армен москва совместный раб

In [3]:
data_df = pd.DataFrame({'comment':text})

data_df.to_csv('\\data.csv')

data_df

,comment
0,поиск ответ начинаться исследование интервью о...
1,"годный однако ,"
2,"офигенный ,"
3,"прекрасный клип додо хороший пицца ,"
4,"отличный ролл ,"
5,"круто ,"
6,"прекрасно удача ,"
7,чело темный тема отвечать ж печально уже лампа...
8,"ороший ролик вкусный пицца ,"
9,"додо родной обожать любить весь сердце ,"


In [4]:
positive = pd.read_csv('russian_sentiment_tweet_automated_labeled/positive.csv',
                       sep = ';',
                       header= None
                      )

negative = pd.read_csv('russian_sentiment_tweet_automated_labeled/positive.csv',
                       sep = ';',
                       header= None
                      )

In [5]:
positive_text = pd.DataFrame(positive.iloc[:, 3])
negative_text = pd.DataFrame(negative.iloc[:, 3])

In [6]:
positive_text['label'] = [1] * positive_text.shape[0]
negative_text['label'] = [0] * negative_text.shape[0]

In [7]:
labeled_tweets = pd.concat([positive_text, negative_text])

In [8]:
labeled_tweets.index = range(labeled_tweets.shape[0])

In [9]:
labeled_tweets.columns = ['text', 'label']
labeled_tweets

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1
...,...,...
229817,"Спала в родительском доме, на своей кровати......",0
229818,RT @jebesilofyt: Эх... Мы немного решили сокра...,0
229819,"Что происходит со мной, когда в эфире #proacti...",0
229820,"""Любимая,я подарю тебе эту звезду..."" Имя како...",0


# О чистка размеченного датасета

In [10]:
# Оставим в тексте только кириллические символы
def clear_text(text):
    clear_text = re.sub(r'[^А-яЁё]+', ' ', text).lower()
    return " ".join(clear_text.split())
    

# напишем функцию удаляющую стоп-слова
def clean_stop_words(text, stopwords):
    text = [word for word in text.split() if word not in stopwords]
    return " ".join(text)

In [11]:
# загрузим список стоп-слов
stopwords = set(nltk_stopwords.words('russian'))
np.array(stopwords)

array({'ну', 'вот', 'ее', 'по', 'чтобы', 'тот', 'при', 'потому', 'но', 'этот', 'уж', 'этого', 'нас', 'эти', 'во', 'к', 'более', 'он', 'том', 'ты', 'три', 'надо', 'была', 'вы', 'них', 'или', 'чем', 'кто', 'а', 'другой', 'его', 'иногда', 'даже', 'может', 'она', 'здесь', 'свою', 'от', 'и', 'когда', 'никогда', 'сам', 'нельзя', 'тем', 'не', 'над', 'после', 'то', 'эту', 'теперь', 'потом', 'чего', 'вас', 'себе', 'был', 'без', 'их', 'почти', 'ведь', 'этой', 'меня', 'для', 'ним', 'ж', 'мы', 'впрочем', 'из', 'они', 'как', 'вдруг', 'разве', 'тебя', 'наконец', 'будет', 'себя', 'до', 'есть', 'ей', 'моя', 'конечно', 'ли', 'что', 'нее', 'все', 'же', 'вам', 'тогда', 'куда', 'хоть', 'чуть', 'о', 'тут', 'были', 'с', 'под', 'того', 'ничего', 'да', 'еще', 'им', 'уже', 'про', 'хорошо', 'какая', 'было', 'всего', 'на', 'всю', 'ему', 'раз', 'лучше', 'об', 'нибудь', 'через', 'тоже', 'бы', 'совсем', 'нет', 'этом', 'быть', 'со', 'один', 'всех', 'такой', 'всегда', 'сейчас', 'зачем', 'опять', 'мой', 'перед', 'мног

In [12]:
# Протестируем работу функции очистки текста
text = labeled_tweets['text'][np.random.randint(labeled_tweets.shape[0])]
print(text)
print('=======================================')
print(clean_stop_words((clear_text(text)), stopwords))

Нелли, сколько можно с ней смеяться!крутая девчонка)
нелли сколько смеяться крутая девчонка


In [13]:
start_clean = time.time()

labeled_tweets['text_clear'] = labeled_tweets['text'].apply(lambda x: clean_stop_words(clear_text(str(x)), stopwords))

print('Обработка текстов заняла: '+str(round(time.time() - start_clean, 2))+' секунд')

Обработка текстов заняла: 2.43 секунд


In [14]:
labeled_tweets = labeled_tweets[['text_clear', 'label']]
labeled_tweets

,text_clear,label
0,школота поверь самое общество профилирующий пр...,1
1,таки немного похож мальчик равно,1
2,идиотка испугалась,1
3,углу сидит погибает голода ещё порции взяли хо...,1
4,значит страшилка блин посмотрев части создастс...,1
...,...,...
229817,спала родительском доме своей кровати проснула...,0
229818,эх немного решили сократить путь лежу мягкой к...,0
229819,происходит мной эфире звучит любимая песня,0
229820,любимая подарю тебе звезду имя звезды переводи...,0


In [15]:
labeled_tweets.to_csv('labeled_tweets_clean.csv')

# Загрузка размеченного датасета Linis Crowd

In [16]:
#labeled_tweets['text_clear'] = labeled_tweets[['text_clear']]
joned_text  = labeled_tweets

In [17]:
joned_text.label.value_counts()

1    114911
0    114911
Name: label, dtype: int64

In [18]:
joned_text.columns = [ 'lemm_text', 'label']

In [19]:
joned_text.index = range(joned_text.shape[0])

joned_text.isna().sum()

# Лемматизация текстов

In [20]:
selected = joned_text
selected

,lemm_text,label
0,школота поверь самое общество профилирующий пр...,1
1,таки немного похож мальчик равно,1
2,идиотка испугалась,1
3,углу сидит погибает голода ещё порции взяли хо...,1
4,значит страшилка блин посмотрев части создастс...,1
...,...,...
229817,спала родительском доме своей кровати проснула...,0
229818,эх немного решили сократить путь лежу мягкой к...,0
229819,происходит мной эфире звучит любимая песня,0
229820,любимая подарю тебе звезду имя звезды переводи...,0


In [21]:
start_clean = time.time()

selected['text_clear'] = selected['lemm_text']\
                               .apply(lambda x:
                                      clean_stop_words(
                                        clear_text(str(x)),
                                        stopwords))

print('Обработка текстов заняла: '+str(round(time.time() - start_clean, 2))+' секунд')

Обработка текстов заняла: 2.11 секунд


In [22]:
def lemmatize(df : (pd.Series, pd.DataFrame),
              text_column : (None, str),
              n_samples : int,
              break_str = 'br',
             ) -> pd.Series:
    """
    Принимает:
    df -- таблицу или столбец pandas содержащий тексты,
    text_column -- название столбца указываем если передаем таблицу,
    n_samples -- количество текстов для объединения,
    break_str -- символ разделения, нужен для ускорения,
    количество текстов записанное в n_samples объединяется 
    в одит большой текст с предварительной вставкой символа 
    записанного в break_str между фрагментами
    затем большой текст лемматизируется, после чего разбивается на
    фрагменты по символу break_str
    
    
    Возвращает:
    Столбец pd.Series с лемматизированными текстами
    в которых все слова приведены к изначальной форме:
    * для существительных — именительный падеж, единственное число;
    * для прилагательных — именительный падеж, единственное число,
    мужской род;
    * для глаголов, причастий, деепричастий — глагол в инфинитиве 
    (неопределённой форме) несовершенного вида.
    
    """
    
    result = []
    
    m = Mystem()    
    
    for i in tqdm(range((df.shape[0] // n_samples) + 1)) :
        
        start = i * n_samples
        
        stop = start + n_samples
        
        sample = break_str.join(df[text_column][start : stop].values)
        
        lemmas = m.lemmatize(sample)
        
        lemm_sample = ''.join(lemmas).split(break_str)
        
        result += lemm_sample
        
    return pd.Series(result, index = df.index)

In [23]:
selected['lemm_clean_tex'] = lemmatize(
    df = selected,
    text_column = 'text_clear',
    n_samples = 100,
    break_str = 'br',
    )

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2299/2299 [52:58<00:00,  1.38s/it]


In [26]:
selected.head()

,lemm_text,label,text_clear,lemm_clean_tex
0,школота поверь самое общество профилирующий пр...,1,школота поверь самое общество профилирующий пр...,школоть поверять самый общество профилировать ...
1,таки немного похож мальчик равно,1,таки немного похож мальчик равно,таки немного похожий мальчик равно
2,идиотка испугалась,1,идиотка испугалась,идиотка испугалась
3,углу сидит погибает голода ещё порции взяли хо...,1,углу сидит погибает голода ещё порции взяли хо...,углу сидеть погибать голод еще порция взять хо...
4,значит страшилка блин посмотрев части создастс...,1,значит страшилка блин посмотрев части создастс...,значит страшилка блин посмотреть часть создава...


In [33]:
sample_1= selected[['lemm_text','lemm_clean_tex','label']]
sample_1

,lemm_text,lemm_clean_tex,label
0,школота поверь самое общество профилирующий пр...,школоть поверять самый общество профилировать ...,1
1,таки немного похож мальчик равно,таки немного похожий мальчик равно,1
2,идиотка испугалась,идиотка испугалась,1
3,углу сидит погибает голода ещё порции взяли хо...,углу сидеть погибать голод еще порция взять хо...,1
4,значит страшилка блин посмотрев части создастс...,значит страшилка блин посмотреть часть создава...,1
...,...,...,...
229817,спала родительском доме своей кровати проснула...,спала родительский дом свой кровать просыпатьс...,0
229818,эх немного решили сократить путь лежу мягкой к...,эх немного решать сокращать путь лежать мягкий...,0
229819,происходит мной эфире звучит любимая песня,происходит я эфир звучать любить песня,0
229820,любимая подарю тебе звезду имя звезды переводи...,любимая подарить ты звезда имя звезда переводи...,0


# Получение TF-IDF векторных представлений размеченных текстов

Для обучения классификатора получим значения IDF (inference document frequency) для слов из тренировочного набора данных, значения IDF равны логарифму отношения количества документов к количеству документов содержащих искомое слово. Например для стандартных слов, которые встречаются практически в любом тексте IDF будет близок к единице, а для специфичных, которые встречаются в одном тексте из 100 это значение будет равно уже 2 (если мы берем основание логарифма 10).


Затем получив словарь со значениями IDF мы можем получить векторное представление каждого текста по следующему принципу, значения IDF слова умножаем на значения

In [34]:
#sample_1 = joned_text
#sample_1.columns = [ 'text_lemm', 'label']
#sample_1["text_clear"] = labeled_tweets[['text_clear']]
sample_1

,lemm_text,lemm_clean_tex,label
0,школота поверь самое общество профилирующий пр...,школоть поверять самый общество профилировать ...,1
1,таки немного похож мальчик равно,таки немного похожий мальчик равно,1
2,идиотка испугалась,идиотка испугалась,1
3,углу сидит погибает голода ещё порции взяли хо...,углу сидеть погибать голод еще порция взять хо...,1
4,значит страшилка блин посмотрев части создастс...,значит страшилка блин посмотреть часть создава...,1
...,...,...,...
229817,спала родительском доме своей кровати проснула...,спала родительский дом свой кровать просыпатьс...,0
229818,эх немного решили сократить путь лежу мягкой к...,эх немного решать сокращать путь лежать мягкий...,0
229819,происходит мной эфире звучит любимая песня,происходит я эфир звучать любить песня,0
229820,любимая подарю тебе звезду имя звезды переводи...,любимая подарить ты звезда имя звезда переводи...,0


In [35]:
# предварительно разделим выборку на тестовую и обучающую
train, test = train_test_split(sample_1,
                        test_size = 0.2,
                        random_state = 12348,
                       )

print(train.shape)
print(test.shape)

(183857, 3)
(45965, 3)


In [36]:
train

,lemm_text,lemm_clean_tex,label
162750,норм понедельник сами,норм понедельник сами,0
55944,откажется последний год дотерпит грозятся,откажется последний год дотерпеть грозятся,1
177605,возму руки тебе станет легче,возму рука ты становиться легче,0
218591,поводу заминирования станций метро вромайдан,поводу заминирование станция метро вромайдан,0
140799,ололо геи китая пронумерованы двоичной системе,ололо гей китай пронумеровывать двоичный систе...,0
...,...,...,...
141717,знаешь братуха хотел просто поиграть любовь вл...,знаешь братуха хотеть просто поиграть любовь в...,0
207453,вспоминаются сразу журналы фрестьянка хоботница,вспоминаются сразу журнал фрестьянка хоботница,0
36863,председатель правительства рф дмитрий медведев...,председатель правительство рф дмитрий медведев...,1
146555,самовьющиеся локоны таинственныйвзгляд,самовьющиеся локон таинственныйвзгляд,0


In [37]:
# Сравним распределение целевого признака
for sample in [train, test]:    
    print(sample[sample['label'] == 1].shape[0] / sample.shape[0])

0.5002148408817723
0.49914065049494183


#Предварительное обучение моделей

In [ ]:

# Получим векторные представления текстов
count_idf_1 = TfidfVectorizer(ngram_range=(1,1))

In [ ]:
tf_idf_base_1 = count_idf_1.fit(sample_1['lemm_text'])
tf_idf_train_base_1 = count_idf_1.transform(train['lemm_text'])
tf_idf_test_base_1 = count_idf_1.transform(test['lemm_text'])

In [ ]:
display(tf_idf_test_base_1.shape)
display(tf_idf_train_base_1.shape)

In [ ]:
model_lr_base_1 = LogisticRegression(solver = 'lbfgs',
                                    random_state = 12345,
                                    max_iter= 10000,
                                     n_jobs=-1)
print('finish')

In [ ]:
model_lr_base_1.fit(tf_idf_train_base_1, train['label'])


In [ ]:
predict_lr_base_proba = model_lr_base_1.predict_proba(tf_idf_test_base_1)
predict_lr_base_proba

In [ ]:
# Выведем 5 случайных комментариев c оценкой негатива первого видео
for _ in range(5):

    source = data_df.sample(n=1)
    text_clear = source['comment'].values[0]
    text = source['comment'].values[0]

    print(text)

    tf_idf_text = count_idf_1.transform([text_clear])

    toxic_proba = model_lr_base_1.predict_proba(tf_idf_text)

    print('Вероятность негатива: ', toxic_proba[:, 0])
    print()

In [41]:
tf_idf_base_1 = count_idf_1.fit(sample_1['lemm_text'])
tf_idf_train_base_1 = count_idf_1.transform(train['lemm_text'])
tf_idf_test_base_1 = count_idf_1.transform(test['lemm_text'])

In [42]:
display(tf_idf_test_base_1.shape)
display(tf_idf_train_base_1.shape)

(45965, 119363)

(183857, 119363)

In [43]:
model_lr_base_1 = LogisticRegression(solver = 'lbfgs',
                                    random_state = 12345,
                                    max_iter= 10000,
                                     n_jobs=-1)
print('finish')

finish


In [47]:
model_lr_base_1.fit(tf_idf_train_base_1, train['label'])


LogisticRegression(max_iter=10000, n_jobs=-1, random_state=12345)

In [48]:
predict_lr_base_proba = model_lr_base_1.predict_proba(tf_idf_test_base_1)
predict_lr_base_proba

array([[0.52108804, 0.47891196],
       [0.59789269, 0.40210731],
       [0.40166341, 0.59833659],
       ...,
       [0.59536057, 0.40463943],
       [0.5474012 , 0.4525988 ],
       [0.51642207, 0.48357793]])

In [49]:
# Выведем 5 случайных комментариев c оценкой негатива первого видео
for _ in range(5):

    source = data_df.sample(n=1)
    text_clear = source['comment'].values[0]
    text = source['comment'].values[0]

    print(text)

    tf_idf_text = count_idf_1.transform([text_clear])

    toxic_proba = model_lr_base_1.predict_proba(tf_idf_text)

    print('Вероятность негатива: ', toxic_proba[:, 0])
    print()

жаль качество ингридиент смочь удерживать уровень тут понятно что дело чизкейк любимый у ,
Вероятность негатива:  [0.46821061]

офигенный ,
Вероятность негатива:  [0.41622328]

очень классный ролик задумываться будущее полнометражный картина ,
Вероятность негатива:  [0.52943401]

додо любить пицца вкусный спасибо готовить я для ,
Вероятность негатива:  [0.49129379]

обожать додо пицца какой город не приезжать первый дело искать ваш ресторан крутой ,
Вероятность негатива:  [0.5341462]

